In [16]:
import numpy as np
import phate
import sys
sys.path.append('../../src/')
from data_script import hemisphere_data, sklearn_swiss_roll
from data_convert import convert_data
from plotly3d.plot import scatter
from negative_sampling import add_negative_samples, make_hi_freq_noise
import pathlib
from scipy.spatial.distance import pdist, squareform
from sklearn.model_selection import train_test_split
import phate
import numpy as np
from scipy.spatial.distance import pdist, squareform
from omegaconf import DictConfig, OmegaConf
import hydra
from toy_data import * 

Generate data

In [17]:
# gt_X, X, _ = sklearn_swiss_roll(n_samples=1000, noise=0.0)
# x = X
# x2 = x.copy()
# x2[:,1] = x[:,1] + x[:,1].max()
# x3 = np.r_[x,x2]
# x2 = x3.copy()
# x2[:,1] = x3[:,1] + x3[:,1].max()
# x3 = np.r_[x3,x2]
# points = x3

In [18]:
# set parameters
length_phi = 15   #length of swiss roll in angular direction
length_Z = 15     #length of swiss roll in z direction
sigma = 0.1       #noise strength
m = 2000         #number of samples

# create dataset
phi = length_phi*np.random.rand(m)
xi = np.random.rand(m)
Z = length_Z*np.random.rand(m)
X = 1./6*(phi + sigma*xi)*np.sin(phi)
Y = 1./6*(phi + sigma*xi)*np.cos(phi)

points = np.array([X, Y, Z]).transpose()


In [19]:
scatter(points)

In [20]:
pts_rot, rotation_matrix = rotate_data(points, return_rot_mat=True, n=50)
points_noisy = add_gaussian_noise(pts_rot, std=0.05)
points_noisy = add_uniform_dropout(points_noisy, low=0.8)
# points_noisy = add_dropout_noise(points_noisy)

In [21]:
def convert_data(X, seed=42, test_size=0.2, knn=5, t=30, n_components=3, decay=40):
    # phate_op = phate.PHATE(random_state=42, knn=20, t=30, n_components=3, decay=5)
    phate_op = phate.PHATE(random_state=seed, t=t, n_components=n_components, knn=knn, decay=decay)
    phate_data = phate_op.fit_transform(X)
    colors = np.zeros(X.shape[0])
    dists_all = squareform(pdist(phate_op.diff_potential))
    ids = np.arange(0, X.shape[0])
    id_train, id_test = train_test_split(ids, test_size=test_size, random_state=seed)
    is_train = np.isin(ids, id_train)
    X_train = X[is_train]
    # phate_op = phate.PHATE(random_state=seed, knn=20, t=30, n_components=3, decay=5)
    phate_op = phate.PHATE(random_state=seed, t=t, n_components=n_components, knn=knn, decay=decay)
    _ = phate_op.fit_transform(X_train)
    dists = dists_all.copy()
    dists_train = squareform(pdist(phate_op.diff_potential))
    dists[is_train][:,is_train] = dists_train
    return dict(
        data=X,
        colors=colors,
        dist=dists,
        dist_all=dists_all, # deprecated.
        phate=phate_data,
        is_train=is_train
    )

In [22]:
data_dict0 = convert_data(points_noisy, knn=10, t=60, decay=40)

Calculating PHATE...
  Running PHATE on 2000 observations and 50 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...


    Calculated KNN search in 0.40 seconds.
    Calculating affinities...
    Calculated affinities in 0.04 seconds.
  Calculated graph and diffusion operator in 0.50 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 5.78 seconds.
  Calculating metric MDS...


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning:

The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.



  Calculated metric MDS in 58.12 seconds.
Calculated PHATE in 64.47 seconds.
Calculating PHATE...
  Running PHATE on 1600 observations and 50 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.24 seconds.
    Calculating affinities...
    Calculated affinities in 0.01 seconds.
  Calculated graph and diffusion operator in 0.26 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 2.51 seconds.
  Calculating metric MDS...


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning:

The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.



  Calculated metric MDS in 33.83 seconds.
Calculated PHATE in 36.62 seconds.


In [23]:
scatter(data_dict0['phate'], s=5, rescale=True)

In [24]:
scatter(data_dict0['data'], s=5)

In [25]:
data_name = 'swiss_roll_wider_hi_dim'
np.savez(f"../../data/{data_name}.npz", **data_dict0)

In [26]:
seed = 32
data_name = 'swiss_roll_sklearn'
noise_types = ['hi-freq-no-add']
noise_type = 'hi-freq-no-add'
noise_level = 1.
mask_dist = False
data_dict = add_negative_samples(data_dict0.copy(), subset_rate=1., noise_rate=noise_level, seed=seed, noise=noise_type, mask_dists=mask_dist, shell=True)
save_folder = f'../../data/negative_sampling_toy_hi_dim/{mask_dist}/'
pathlib.Path(save_folder).mkdir(parents=True, exist_ok=True)
np.savez(f"{save_folder}/{data_name}.npz", **data_dict)
scatter(data_dict['data'], data_dict['mask_x'], s=2)